In [1]:
import re
import os
import json
import itertools
import pandas as pd
pd.set_option('display.max_colwidth', 1000)

In [2]:
filename_list = os.listdir("./data/posts_comments/")
data_list = []
for i in filename_list:
    if i not in [".ipynb_checkpoints", ".DS_Store"]:
        try:
            data_list.append(json.load(open("./data/posts_comments/"+i,"r")))
        except:
            print(f"error {i}")

In [3]:
f = open("./data/exclude_link.txt", "r")
exclude_post_links = f.readlines()
exclude_post_links = [link.replace("\n","" ) for link in exclude_post_links]
data_list = [post for post in data_list if post['post_url'] not in exclude_post_links]

In [4]:
post_df = pd.DataFrame()
for col in ['post_url','post_time','post_content','link_title']:
    post_df[col] = [post.get(col,None) for post in data_list]

post_df['total_reaction'] = [post.get('reaction_stat',dict()).get('total', None) for post in data_list]
post_df['total_reaction'] = post_df['total_reaction'].apply(lambda x:float(x.replace('K',""))*1000 if 'K' in x else int(x))
post_df['top_3_reaction'] = [post.get('reaction_stat',dict()).get('top_3_reaction', None) for post in data_list]
post_df['top_3_reaction'] = post_df['top_3_reaction'].apply(lambda x:{i.split(":")[0]:i.split(":")[-1] for i in x})
post_df = pd.concat([post_df, post_df['top_3_reaction'].apply(pd.Series)],axis='columns')
for col in ['Like','Haha','Angry','Wow','Love','Sad','Care']:
    post_df[col] = post_df[col].fillna('0 people')
    post_df[col] = post_df[col].str.replace("people","")
    post_df[col] = post_df[col].str.replace("person","")
    post_df[col] = post_df[col].apply(lambda x:float(x.replace("K",""))*1000 if "K" in x else int(x))
post_df = post_df.drop(['top_3_reaction'],axis='columns')
post_df['fanpage'] = post_df['post_url'].apply(lambda x:x.split("/posts")[0].split("/")[-1])

In [5]:
post_df.head()

,post_url,post_time,post_content,link_title,total_reaction,Like,Haha,Angry,Wow,Love,Sad,Care,fanpage
0,https://www.facebook.com/ETtoday/posts/3576468832389255,"July 18, 2020",綁定電子，連紙都省了\n（#丹利老爹）\n●快下載新聞雲App掌握政治大小事\nhttp://goo.gl/sJmzjp,「塑膠片」固定三倍券！網嗆「到底是誰想的」：不能用紙夾嗎？ | ETtoday新聞雲,1700.0,1300.0,288.0,35.0,0.0,0,0,0,ETtoday
1,https://www.facebook.com/ETtoday/posts/3570621259640679,"July 16, 2020",各位家長們怎麼看呢？？（#漁紋熱 \n）\n●快下載新聞雲App掌握社會大小事\nhttp://goo.gl/sJmzjp,領1萬2三倍券！單親媽遭國二孩討錢回絕竟被嗆「侵占」...網友戰翻 | ETtoday新聞雲,729.0,531.0,114.0,0.0,66.0,0,0,0,ETtoday
2,https://www.facebook.com/tvbsfb/posts/10157527479600933,"July 29, 2020",5.4億印製成本？！！\n下載食尚玩家APP！投票抽住宿券→https://bit.ly/302HnCl\n【經濟部認了！3倍券光印製就砸5.4億「多消費券2倍」】\nhttps://news.tvbs.com.tw/life/1361378?from=fbarticle,經濟部認了！3倍券光印製就砸5.4億「多消費券2倍」│TVBS新聞網,12000.0,4800.0,1700.0,4500.0,0.0,0,0,0,tvbsfb
3,https://www.facebook.com/YahooTWNews/posts/3499216643460610,"July 13, 2020",網友大嘆：「無法理解想便民為何做半套？」\n#三倍券 #簡訊 #領取 #YDD,他收三倍券簡訊氣炸 網：唐鳳寫的？ - Yahoo奇摩新聞,920.0,704.0,132.0,51.0,0.0,0,0,0,YahooTWNews
4,https://www.facebook.com/news.ebc/posts/4301458776555847,"May 25, 2020",付1千領3千 人人皆可領！政院砸400億發振興券\n#看我這編：大家覺得這次的振興券如何呢\n#行政院 #振興券 #消費,付1千領3千 人人皆可領！政院砸400億發振興券｜東森財經新聞,2200.0,1600.0,180.0,371.0,0.0,0,0,0,news.ebc


In [6]:
comment_df = pd.DataFrame()
for post in data_list:
    post_url = post.get('post_url', None)
    comments = post.get('comments', [])
    if comments:
        comments = [[comment.get('main')]+comment.get('replies',[]) for comment in comments]
        comments = list(itertools.chain.from_iterable(comments))
        comments = [i.replace("Top Fan\n","") if i.startswith("Top Fan\n") else i for i in comments]
    post_comment = pd.DataFrame(comments,columns=['comment'])
    post_comment['post_url'] = post_url
    comment_df = pd.concat([comment_df, post_comment],axis='rows')
comment_df['author'] = comment_df['comment'].apply(lambda x:x.split("\n")[0])
comment_df['comment'] = comment_df['comment'].apply(lambda x:"\n".join(x.split("\n")[1:]))
comment_df['reaction_num'] = comment_df['comment'].apply(lambda x:int(x.split("\n")[-1]) if re.search("\\n\d+$",x) else 0)
comment_df['comment'] = comment_df['comment'].apply(lambda x:re.sub("\\n\d+","",x))
comment_df.reset_index(inplace=True,drop=True)
comment_df = comment_df[['post_url','author','comment','reaction_num']]

In [7]:
comment_df.head()

,post_url,author,comment,reaction_num
0,https://www.facebook.com/ETtoday/posts/3576468832389255,林皓瑋,昨天我拿到也傻眼 製作的越複雜越多油水可以撈,0
1,https://www.facebook.com/ETtoday/posts/3576468832389255,Linda Luo,林皓瑋 綁定電子就可以讓政府少撈一點油水哦⋯⋯啊。你拿了紙本嗎,9
2,https://www.facebook.com/ETtoday/posts/3576468832389255,Jaime Zhou,林皓瑋 總製作量也沒有第三公正單位查核，也不知道總數量多少？嘿嘿…,3
3,https://www.facebook.com/ETtoday/posts/3576468832389255,百聰林,Jaime Zhou有關係就沒關係 頂多就超用 然後就升官發財 哈哈,1
4,https://www.facebook.com/ETtoday/posts/3576468832389255,Chien-Hsun Chen,林哲誼 你這邏輯跟明明有發票載具還硬要拿紙本然後靠北不環保,2


In [8]:
comment_df.shape

(218134, 4)

In [9]:
posts_picked = pd.read_excel("./data/refined/posts_v1.xlsx", engine='openpyxl')
posts_picked = posts_picked[posts_picked['post_url'].isin(exclude_post_links)==False]

In [10]:
def phase(x):
    if x<pd.to_datetime('2020-01-31'):
        return 0
    elif x <pd.to_datetime('2020-06-02'):
        return 1
    elif x<pd.to_datetime('2020-07-15'):
        return 2
    elif x<pd.to_datetime('2021-01-01'):
        return 3

In [11]:
posts_picked['post_time'] = pd.to_datetime(posts_picked['post_time'])
posts_picked['phase'] = posts_picked['post_time'].apply(phase)

In [12]:
comment_df = comment_df[comment_df['post_url'].isin(posts_picked['post_url'].unique())]

In [13]:
comment_df.shape

(218122, 4)

In [14]:
# # save to files
# post_df.to_csv("./data/refined/posts.csv",index=None)
posts_picked.to_csv("./data/refined/posts_v1.csv", index=None)
comment_df.to_csv("./data/refined/comments.csv", index=None)